## 数据检索
用户输入只是问题，答案需要从已知的数据库或者文档中获取。因此，我们需要使用检索器获取上下文，并通过“question”键下的用户输入。

### RAG
检索增强生成（Retrieval-augmented Generation，RAG），是当下最热门的大模型前沿技术之一。如果将“微调（finetune）”理解成大模型内化吸收知识的过程，那么RAG就相当于给大模型装上了“知识外挂”，基础大模型不用再训练即可随时调用特定领域知识。

In [37]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_path = 'D:\\AInewModels\\BAAI\\bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)

In [40]:
vectorstore = FAISS.from_texts(
    ["小明在华为工作","熊喜欢吃蜂蜜","我喜欢种花"],
    embedding=embeddings
)
vectorstore



In [41]:
#使用向量数据库生成检索器
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [42]:
retriever.invoke("熊喜欢吃什么？")

[Document(id='18c7b63c-b0b7-424c-b861-18fd231e28c5', metadata={}, page_content='熊喜欢吃蜂蜜'),
 Document(id='c8517df3-7035-4af7-aab2-343385a86dd4', metadata={}, page_content='我喜欢种花')]

In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"score_threshold": 0.7})

In [30]:
retriever.invoke("小明在哪里工作？")

[Document(id='68f82581-5c6c-45be-a6da-f641b679f291', metadata={}, page_content='小明在华为工作')]

In [ ]:
help( vectorstore.as_retriever)

In [31]:
from langchain_openai import ChatOpenAI, OpenAI
API_KEY="sk-kEjwwIwe4w4EhkfGei4kNkawKIZeyEfezSdLfiyJrR4jdgPl"
BASE_URL = "https://api.fe8.cn/v1"
MODEL= "gpt-4o-mini"
EMBEDDING_MODEL ="text-embedding-3-small"

model = ChatOpenAI(
    model=MODEL,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL,
    temperature=0.3,
)


In [33]:
# 1.建立Prompt
from langchain_core.prompts import ChatPromptTemplate

template ="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n只根据以下文档回答问题：\n{context}\n\n问题：{question}\n'), additional_kwargs={})])

In [23]:
# 2。建setup_and_retrieval和outputParser
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

outputParser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {
        "context":retriever,
        "question":RunnablePassthrough()
    }
)

chain = setup_and_retrieval | prompt | model | outputParser

In [ ]:
help(RunnableParallel)

In [24]:
chain.invoke("小明在哪里工作？")

'小明在华为工作。'

In [34]:
#完整链式（直接使用链式)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("小明在哪里工作？")

'小明在华为工作。'

In [35]:
from operator import itemgetter

template ="""
只根据以下文档回答问题：
{context}

问题：{question}
回答问题请加上称呼"{name}"。
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,   #将question也要传给retriever
        "question": itemgetter("question"),
        "name": itemgetter("name"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question":"小明在哪里工作？","name":"主人"})

'主人，小明在华为工作。'